In [1]:
!pip install --upgrade plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d arjunprasadsarkhel/2021-olympics-in-tokyo
!unzip 2021-olympics-in-tokyo.zip

mkdir: cannot create directory ‘/root/.kaggle’: File exists
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
unzip:  cannot find or open 2021-olympics-in-tokyo.zip, 2021-olympics-in-tokyo.zip.zip or 2021-olympics-in-tokyo.zip.ZIP.


In [3]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pyspark

spark = pyspark.sql.SparkSession(pyspark.SparkContext())
spark

In [5]:
!pip install -U kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import pandas as pd

def excel_to_csv(filename):
  pd.read_excel(filename + '.xlsx').to_csv(filename + '.csv', sep=',', index = False)

excel_to_csv('Athletes')
excel_to_csv('Coaches')
excel_to_csv('EntriesGender')
excel_to_csv('Medals')
excel_to_csv('Teams')

/usr/local/lib/python3.7/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [7]:
athletes = pd.read_csv('Athletes.csv')
coaches = pd.read_csv('Coaches.csv')
entries_gender = pd.read_csv('EntriesGender.csv')
medals = pd.read_csv('Medals.csv')
teams = pd.read_csv('Teams.csv')
athletes.head()

,Name,NOC,Discipline
0,AALERUD Katrine,Norway,Cycling Road
1,ABAD Nestor,Spain,Artistic Gymnastics
2,ABAGNALE Giovanni,Italy,Rowing
3,ABALDE Alberto,Spain,Basketball
4,ABALDE Tamara,Spain,Basketball


In [8]:
medals.head()

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1
1,2,People's Republic of China,38,32,18,88,2
2,3,Japan,27,14,17,58,5
3,4,Great Britain,22,21,22,65,4
4,5,ROC,20,28,23,71,3


In [9]:
medals.sort_values('Rank by Total', inplace = True)
medals.head()

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1
1,2,People's Republic of China,38,32,18,88,2
4,5,ROC,20,28,23,71,3
3,4,Great Britain,22,21,22,65,4
2,3,Japan,27,14,17,58,5


In [10]:
import plotly.express as px

fig = px.bar(medals, x = 'Team/NOC', y=['Gold', 'Silver', 'Bronze'], color_discrete_sequence =['Gold', 'Silver', 'Brown'] ,title = 'Number of medals per country')
fig.show()

In [11]:
fig.write_image("images/medals.png")

In [12]:
coaches.head()

,Name,NOC,Discipline,Event
0,ABDELMAGID Wael,Egypt,Football,NaN
1,ABE Junya,Japan,Volleyball,NaN
2,ABE Katsuhiko,Japan,Basketball,NaN
3,ADAMA Cherif,Côte d'Ivoire,Football,NaN
4,AGEBA Yuya,Japan,Volleyball,NaN


In [13]:
spark.read.csv('Coaches.csv', header = True, inferSchema=True).createOrReplaceTempView('Coaches')
spark.sql('Select * from Coaches').show(5)

+---------------+-------------+----------+-----+
|           Name|          NOC|Discipline|Event|
+---------------+-------------+----------+-----+
|ABDELMAGID Wael|        Egypt|  Football| null|
|      ABE Junya|        Japan|Volleyball| null|
|  ABE Katsuhiko|        Japan|Basketball| null|
|   ADAMA Cherif|Côte d'Ivoire|  Football| null|
|     AGEBA Yuya|        Japan|Volleyball| null|
+---------------+-------------+----------+-----+
only showing top 5 rows



In [14]:
pd_coaches_query = spark.sql('Select NOC, COUNT(NOC) as count from Coaches group by NOC order by NOC').toPandas()
pd_coaches_query.head(7)

,NOC,count
0,Angola,1
1,Argentina,11
2,Australia,22
3,Austria,3
4,Bahrain,1
5,Belarus,1
6,Belgium,5


In [15]:
fig = px.bar(pd_coaches_query, x = 'NOC', y = 'count', color = 'count', title = 'Number of coaches from each country')
fig.show()

In [16]:
fig.write_image("images/coaches.png")

In [17]:
athletes.head()

,Name,NOC,Discipline
0,AALERUD Katrine,Norway,Cycling Road
1,ABAD Nestor,Spain,Artistic Gymnastics
2,ABAGNALE Giovanni,Italy,Rowing
3,ABALDE Alberto,Spain,Basketball
4,ABALDE Tamara,Spain,Basketball


In [18]:
spark.read.csv('Athletes.csv', header = True, inferSchema=True).createOrReplaceTempView('Athletes')
spark.sql('select * from athletes order by NOC, Discipline').show()

+--------------------+-----------+-------------------+
|                Name|        NOC|         Discipline|
+--------------------+-----------+-------------------+
|       YOUSOFI Kimia|Afghanistan|          Athletics|
|NOOR ZAHI Sha Mah...|Afghanistan|          Athletics|
|        YOVARI Mahdi|Afghanistan|           Shooting|
|        ANWARI Fahim|Afghanistan|           Swimming|
|     MANSOURI Farzad|Afghanistan|          Taekwondo|
|       PETROV Matvei|    Albania|Artistic Gymnastics|
|       SMAJLAJ Izmir|    Albania|          Athletics|
|      CULLHAJ Indrit|    Albania|               Judo|
|     DELILAJ Manuela|    Albania|           Shooting|
|         KADIU Kledi|    Albania|           Swimming|
|       MERIZAJ Nikol|    Albania|           Swimming|
|        CALJA Briken|    Albania|      Weightlifting|
|      QERIMAJ Erkand|    Albania|      Weightlifting|
|      HETHAT Yassine|    Algeria|          Athletics|
|TRIKI Yasser Mohamed|    Algeria|          Athletics|
| LAHOULOU

In [19]:
spark.sql('''select first(NOC) as NOC, first(Discipline) as Discipline, count(*) as count from athletes group by NOC, Discipline order by NOC, Discipline''').show()

+-----------+-------------------+-----+
|        NOC|         Discipline|count|
+-----------+-------------------+-----+
|Afghanistan|          Athletics|    2|
|Afghanistan|           Shooting|    1|
|Afghanistan|           Swimming|    1|
|Afghanistan|          Taekwondo|    1|
|    Albania|Artistic Gymnastics|    1|
|    Albania|          Athletics|    1|
|    Albania|               Judo|    1|
|    Albania|           Shooting|    1|
|    Albania|           Swimming|    2|
|    Albania|      Weightlifting|    2|
|    Algeria|          Athletics|    8|
|    Algeria|             Boxing|    8|
|    Algeria|       Canoe Sprint|    1|
|    Algeria|       Cycling Road|    2|
|    Algeria|            Fencing|    3|
|    Algeria|               Judo|    2|
|    Algeria|  Marathon Swimming|    1|
|    Algeria|             Rowing|    2|
|    Algeria|            Sailing|    2|
|    Algeria|           Shooting|    1|
+-----------+-------------------+-----+
only showing top 20 rows



In [20]:
pd_athletes_query = spark.sql('''select first(NOC) as NOC, first(Discipline) as Discipline, count(*) as count from athletes group by NOC, Discipline order by NOC, Discipline''').toPandas()

In [21]:
fig = px.bar(pd_athletes_query, x = 'NOC', y = 'count', color='Discipline', title = 'Number of athletes from each country per discipline')
fig.show()

In [23]:
athletes['Discipline'].value_counts().head()

Athletics    2068
Swimming      743
Football      567
Rowing        496
Hockey        406
Name: Discipline, dtype: int64

In [24]:
pd_athletes_query = pd_athletes_query.loc[pd_athletes_query['Discipline'].isin(['Athletics', 'Swimming', 'Football'])]
pd_athletes_query

,NOC,Discipline,count
0,Afghanistan,Athletics,2
2,Afghanistan,Swimming,1
5,Albania,Athletics,1
8,Albania,Swimming,2
10,Algeria,Athletics,8
...,...,...,...
2103,Zambia,Athletics,2
2105,Zambia,Football,21
2107,Zambia,Swimming,2
2108,Zimbabwe,Athletics,1


In [25]:
fig = px.bar(pd_athletes_query, x = 'NOC', y = 'count', color = 'Discipline', title = 'Number of athletes from each country per discipline')
fig.show()

In [26]:
fig.write_image("images/athletes.png")

In [27]:
entries_gender.head()

,Discipline,Female,Male,Total
0,3x3 Basketball,32,32,64
1,Archery,64,64,128
2,Artistic Gymnastics,98,98,196
3,Artistic Swimming,105,0,105
4,Athletics,969,1072,2041


In [28]:
len(entries_gender)

46

In [29]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

specs = [[{'type' : 'domain'}, {'type': 'domain'}]] *  23 

In [30]:
fig = make_subplots(rows = 23, cols = 2, subplot_titles=entries_gender['Discipline'], specs=specs)

sub_figs = []
for index, row in entries_gender.iterrows():
  sub_fig = go.Pie(labels=['Female', 'Male'], values = [row['Female'], row['Male']])
  sub_figs.append(sub_fig)

k = 0
for i in range(1, 24):
  for j in range(1, 3):
    fig.add_trace(sub_figs[k], i, j)
    k += 1

fig.update_layout(showlegend = False, height = 10000, width = 800, title_text = 'Distribution of Gender amongst each game')
fig.update_traces(textposition = 'inside', textinfo = 'label+percent', hoverinfo = 'label+value+percent')

fig = go.Figure(fig)

fig.show()


In [46]:
fig.write_image("images/EntriesGender.png")

In [32]:
teams.head(10)

,Name,Discipline,NOC,Event
0,Belgium,3x3 Basketball,Belgium,Men
1,China,3x3 Basketball,People's Republic of China,Men
2,China,3x3 Basketball,People's Republic of China,Women
3,France,3x3 Basketball,France,Women
4,Italy,3x3 Basketball,Italy,Women
5,Japan,3x3 Basketball,Japan,Men
6,Japan,3x3 Basketball,Japan,Women
7,Latvia,3x3 Basketball,Latvia,Men
8,Mongolia,3x3 Basketball,Mongolia,Women
9,Netherlands,3x3 Basketball,Netherlands,Men


In [37]:
spark.read.csv('Teams.csv', header = True, inferSchema=True).createOrReplaceTempView('Teams')
pd_teams_query = spark.sql('select Name, Count(Discipline) as Count_Discipline, Event from teams group by Event,Name order by Name, Count_Discipline').toPandas()
pd_teams_query.head()

,Name,Count_Discipline,Event
0,ACHANTA Kamal / BATRA Manika,1,Mixed Doubles
1,ASSAR Omar / MESHREF Dina,1,Mixed Doubles
2,Abicha/Elgraoui,1,Men
3,Agatha/Duda,1,Women
4,Alison/Alvaro Filho,1,Men


In [38]:
pd_teams_query['Event'].value_counts().head()

Men             71
Women           60
Mixed Team      29
Men's Team      26
Women's Team    26
Name: Event, dtype: int64

In [42]:
pd_teams_query = pd_teams_query.loc[pd_teams_query['Event'].isin(['Men', 'Women', 'Mixed Team'])]

In [48]:
fig = px.bar(pd_teams_query, x = 'Name', y = 'Count_Discipline', color = 'Event', title = 'Number of Disciplines From Each Team per Event')
fig.show()

In [49]:
fig.write_image("images/Teams.png")